# Online Sorting Demo
In a selected recording, use RT-Sort to 

1) Detect sequences in the first part of the recording (offline mode)
2) Assign spikes in real-time in the second part (online mode)

In [1]:
%load_ext autoreload

In [2]:
# Imports
import os
from pathlib import Path

%autoreload 2
from braindance.core.maxwell_env import MaxwellEnv, stop_process_using_port
from braindance.core.spikesorter.rt_sort import detect_sequences, RTSort, save_traces

No maxlab found, instead using dummy maxlab module!
Cannot import torch_tensorrt


In [3]:
# RT-Sort setup
recording_path = "patch_rec_cell2.raw.h5"
inter_path = "2602"
detection_model_path = "braindance/core/spikedetector/detection_models/mea"
offline_recording_window_ms = (0, 5*60*1000)  # Section of recording for detecting sequences (times in milliseconds)
num_processes=os.cpu_count()//2  # The optimal num_processes for the fastest computation depends on your system
verbose = True  # Whether to print progress

online_recording_window_ms = (5*60*1000, 10*60*1000)  # Section of recording for online sorting 

In [ ]:
# Detect sequences (nothing to change here)
rt_sort = detect_sequences(
    recording=recording_path,
    inter_path=inter_path,
    detection_model=detection_model_path,
    recording_window_ms=offline_recording_window_ms, 
    num_processes=num_processes,
    verbose=verbose
)

In [4]:
# OPTIONAL: Load RT-Sort (helpful if this notebook has been restarted after running the previous cell)
#           RT-Sort is saved at inter_path/rt_sort.pickle
rt_sort = RTSort.load_from_file(Path(inter_path) / "rt_sort.pickle", detection_model_path)

Cannot compile detection model with torch_tensorrt because cannot load torch_tensorrt. Skipping NVIDIA compilation


c:\Users\riceh\anaconda3\envs\brain_dance\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [4]:
# Setup recording for online sorting (nothing to change here)
scaled_traces_path = save_traces(recording_path, Path(inter_path) / "online",
                                *online_recording_window_ms, 
                                num_processes=num_processes, dtype="float32",
                                verbose=verbose)

Saving traces:


100%|██████████| 60/60 [01:07<00:00,  1.13s/it]


In [5]:
# Set up params for online sorting
maxwell_params = {
    # Total duration of online sorting in seconds
    "max_time_sec": 5, # (online_recording_window_ms[1] - online_recording_window_ms[0]) / 1000, 
    
    # An empty folder will be created at this path. For this notebook, this folder can be deleted
    "save_dir": "save_dir",

    # If the computer is not connected to a physical MEA, set "dummy" to path of recording to simulate a real recording. Otherwise, set to None.
    "dummy": Path(inter_path) / "online/scaled_traces.npy",
    
    "config": None,
    "stim_electrodes": [],
    "multiprocess": False,
    "render": False,
    "observation_type": "raw",
}
done = False

# Start online sorting
rt_sort.reset()
env = MaxwellEnv(**maxwell_params)

sequence_detections = [[] for _ in range(rt_sort.num_seqs)]

while not done:
    obs, done = env.step(buffer_size=rt_sort.buffer_size)    
    rt_sort.latest_frame = env.latest_frame - len(obs) + 1
    detections = rt_sort.running_sort(obs)
    for sequence_idx, spike_time in detections:
        print(f"Spike at {spike_time/1000:.4f}s was assigned to sequence {sequence_idx}")
        sequence_detections[sequence_idx].append(spike_time)

No config file specified
Launching dummy server
Using dummy data: 
	 C:\Users\riceh\kosik\data\2602\online\scaled_traces.npy
Could not import smart_open_braingeneers, using smart_open
Successfully ignored first packet
Dummy server ready
Initializing MaxOne
MaxOne initialized
Setting up subscribers
Recording electrodes initialized from: None
Electrodes selected for stimulation: []
Power cycled
Offseting
Successfully ignored first packet
Successfully ignored remaining packets
Name of experiment:  
At  C:\Users\riceh\kosik\data\delete_me\.raw.h5
Recording from 0 channels and 0 stim electrodes
===================== Beginning experiment =====================
Spike at 0.5275s was assigned to sequence 14
Spike at 0.5674s was assigned to sequence 0
Spike at 0.6321s was assigned to sequence 18
Spike at 0.7192s was assigned to sequence 33
Spike at 0.7163s was assigned to sequence 10
Spike at 0.7555s was assigned to sequence 5
Spike at 0.8594s was assigned to sequence 24
Spike at 0.8665s was assi